## Проверка загруженной модели

In [29]:
!ls -la

total 120
drwxrwxr-x 4 ubuntu ubuntu  4096 Dec 20 06:31 .
drwxrwxr-x 3 ubuntu ubuntu  4096 Dec 20 06:07 ..
drwxr-xr-x 3 ubuntu ubuntu  4096 Dec 20 06:22 Dima_flights_LR_only2
-rw-rw-r-- 1 ubuntu ubuntu  2765 Dec 20 06:07 flights_LR_only_solution.py
-rw-rw-r-- 1 ubuntu ubuntu   183 Dec 20 06:07 flights_LR_only_solution.sh
-rw-rw-r-- 1 ubuntu ubuntu 18454 Dec 20 06:31 flights_mlflow-check.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  3934 Dec 20 06:21 flights_pipeline_solution.py
-rw-rw-r-- 1 ubuntu ubuntu   180 Dec 20 06:07 flights_pipeline_solution.sh
-rw-rw-r-- 1 ubuntu ubuntu  5782 Dec 20 06:07 flights_pipe_withHP_solution.py
-rw-rw-r-- 1 ubuntu ubuntu   189 Dec 20 06:07 flights_pipe_withHP_solution.sh
drwxrwxr-x 2 ubuntu ubuntu  4096 Dec 20 06:11 .ipynb_checkpoints
-rw-rw-r-- 1 ubuntu ubuntu 49342 Dec 20 06:07 mlflow-spark-1.27.0.jar
-rw-rw-r-- 1 ubuntu ubuntu   363 Dec 20 06:12 mlflow_test.py


In [8]:
APP_NAME='Flights_Spark_TrainValidationSplit'

In [9]:
S3_DATA='s3a://mlflowbucket/flights_larger.csv'

### Используйте свой путь до директории с моделью

In [28]:
# Модель лежит локально (если в HDFS не найдет, то будет искать локально)
# saved_model_path = 'Dima_flights_LR_only2'

# Модель лежит в S3
saved_model_path = 's3://mlflowbucket/artifacts/1/57428478da124244879f81c3ebaacf78/artifacts/Dima_flights_LR_only2'

In [29]:
! pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [30]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [31]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession

In [32]:
#! pip install mlflow

In [33]:
import mlflow

In [34]:
from datetime import datetime

In [10]:
!ls $HOME/.aws

credentials


In [35]:
spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .config("spark.jars", "mlflow-spark-1.27.0.jar")\
    .getOrCreate()

In [11]:
# spark.stop()

In [36]:
data = (spark.read.format('csv')
        .options(header='true', inferSchema='true')
        .load(S3_DATA))

In [37]:
assembler = VectorAssembler(inputCols=["mile"], outputCol="features")
out_data = assembler.transform(data)

In [38]:
out_data.show()

+---+---+---+-------+------+---+----+------+--------+-----+--------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|features|
+---+---+---+-------+------+---+----+------+--------+-----+--------+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27| [157.0]|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102|   NA| [466.0]|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19| [738.0]|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|[2248.0]|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22| [386.0]|
|  3| 28|  1|     B6|   377|LGA|1076| 13.33|     182|   70|[1076.0]|
|  5| 28|  6|     B6|   904|ORD| 740|  9.58|     130|   47| [740.0]|
|  1| 19|  2|     UA|   820|SFO| 679| 12.75|     123|  135| [679.0]|
|  8|  5|  5|     US|  2175|LGA| 214|  13.0|      71|  -10| [214.0]|
|  5| 27|  5|     AA|  1240|ORD|1197| 14.42|     195|  -11|[1197.0]|
|  8| 20|  6|     B6|   119|JFK|1182| 14.67|     198|   20|[1182.0]|
|  2|  3|  1|     AA|  1881|JFK|10

In [39]:
evaluator = RegressionEvaluator(labelCol='duration')

In [40]:
saved_model_path = 's3://mlflowbucket/artifacts/79cb2d2a17dc48cfb4bd59826a919cf7/artifacts/Dima_flights_LR_only2'
loaded_model = mlflow.spark.load_model(model_uri=saved_model_path)

2022/12/20 11:14:31 INFO mlflow.spark: URI 's3://mlflowbucket/artifacts/79cb2d2a17dc48cfb4bd59826a919cf7/artifacts/Dima_flights_LR_only2/sparkml' does not point to the current DFS.
2022/12/20 11:14:31 INFO mlflow.spark: File 's3://mlflowbucket/artifacts/79cb2d2a17dc48cfb4bd59826a919cf7/artifacts/Dima_flights_LR_only2/sparkml' not found on DFS. Will attempt to upload the file.
2022/12/20 11:14:31 INFO mlflow.spark: Copied SparkML model to /tmp/mlflow/cac8f8c8-d11f-425f-822f-b7690acf6ebc


In [41]:
loaded_model

PipelineModel_d58c881dc114

In [42]:
# Предсказываем на сырых данных, так как в модели заложен функционал предобработки данных
data = (spark.read.format('csv')
        .options(header='true', inferSchema='true')
        .load(S3_DATA))

prediction = loaded_model.transform(data).select("features", "duration", "prediction")

In [43]:
prediction

DataFrame[features: vector, duration: int, prediction: double]

In [44]:
prediction.show()

+--------------------+--------+------------------+
|            features|duration|        prediction|
+--------------------+--------+------------------+
|(8,[0,1],[157.0,1...|      51| 62.97125064283171|
|(8,[0,1],[466.0,1...|     102| 99.92322767816631|
|(8,[0,1],[738.0,1...|     127|132.45053432415665|
|(8,[0,3],[2248.0,...|     365|337.57326889367556|
|(8,[0,6],[386.0,1...|      85| 80.13234215938485|
|(8,[0,4],[1076.0,...|     182| 191.5073855195266|
|(8,[0,1],[740.0,1...|     130|132.68970569655363|
|(8,[0,2],[679.0,1...|     123|117.44261305149053|
|(8,[0,4],[214.0,1...|      71| 88.42452401642484|
|(8,[0,1],[1197.0,...|     195|187.34036428926535|
|(8,[0,3],[1182.0,...|     198| 210.0949274060811|
|(8,[0,3],[1090.0,...|     200|199.09304427581966|
|(8,[0,3],[1028.0,...|     193|191.67873173151307|
|(8,[0,1],[733.0,1...|     125| 131.8526058931642|
|(8,[0,1],[733.0,1...|     129| 131.8526058931642|
|(8,[0,4],[292.0,1...|     102| 97.75220753990737|
|(8,[0,1],[612.0,1...|     109|

In [60]:
evaluator.evaluate(prediction)

11.209003907667439

In [20]:
spark.stop()